# Record Random Views in Simulation

Record random views in simulation. This type of data is easy to collect on a real robot, which is why its a good candidate to use it for pre-training a similarity function. Set `num_episodes` to control the number of scenes recorded and `max_steps` to control the numbers of views recorded.

The equivalent recordings from the real robot can be found here:

`/misc/lmbraid19/argusm/CLUSTER/robot_recordings/pose_estimation/ur3_objects`

In [ ]:
import os
import time
import json
import shutil
import unittest
import subprocess
from pathlib import Path

from scipy.spatial.transform import Rotation as R

from gym_grasping.envs.robot_sim_env import RobotSimEnv
from flow_control.demo.demo_episode_recorder import record_sim
from flow_control.runner import evaluate_control
from flow_control.servoing.module import ServoingModule
from math import pi
import getpass
from robot_io.calibration.gripper_cam_calibration import GripperCamPoseSampler


experiment = "random_views"
goal = "random_views_test"

def get_data_dir():
    username = getpass.getuser()
    if username == "argusm":
        return "/tmp/flow_experiments3"
    elif username == "nayakab":
        return "../tmp"

data_dir = get_data_dir()
root_dir = os.path.join(data_dir, experiment)
goal_dir = os.path.join(data_dir, goal)

renderer = "egl"
object_selected = "trapeze"
task_variant = "rP"  # rotation plus (+-pi)

def get_configurations(root_dir=root_dir, task="shape_sorting", num_episodes=20, start_seed=0, prefix="", object_selected='trapeze'):
    os.makedirs(root_dir, exist_ok=True)
    save_dir_template = os.path.join(root_dir, f"{prefix}_{task}_{object_selected}")
    for seed in range(start_seed, start_seed + num_episodes):
        save_dir = save_dir_template + f"_{task_variant}"+f"_seed{seed:03d}"
        yield object_selected, task_variant, seed, save_dir

In [ ]:
from robot_io.recorder.simple_recorder import SimpleRecorder
from tqdm import tqdm

cfg = { 0: {'prefix': 'train', 'task': 'shape_sorting', 'num_episodes': 200, 'start_seed': 0, 'object_selected': object_selected, 'root_dir': root_dir},
      #  1: {'prefix': 'test', 'task': 'shape_sorting', 'num_episodes': 20, 'start_seed': 1000, 'object_selected': object_selected, 'root_dir': root_dir}
      }

max_steps = 200
for key, value in cfg.items():
    demo_cfg = get_configurations(value['root_dir'], value['task'], 
                                  value['num_episodes'], value['start_seed'],
                                  prefix=value['prefix'],
                                  object_selected=value['object_selected'])
    
    for object_selected, task_variant, seed, save_dir in demo_cfg:
        param_info = {"object_selected": value['object_selected'], "task_selected": value['task']}
        env = RobotSimEnv(task='recombination', renderer=renderer, act_type='continuous',
                          initial_pose='close', max_steps=max_steps, control='absolute-full',
                          img_size=(256, 256),
                          param_randomize=("geom",),
                          param_info=param_info,
                          task_info=dict(object_rot_range={"rP":pi/2.,"rR":pi/6.}[task_variant]),
                          seed=seed)

        if task_variant == "rP":
            assert env.params.variables[f"{object_selected}_pose"]["d"][3] == pi/2.
        elif task_variant == "rR":
            assert env.params.variables[f"{object_selected}_pose"]["d"][3] == pi/6.
        #update_object_orn(env, object_selected, orn)

        s = 2.0
        inital_pos, initial_orn = env.robot.get_tcp_pos_orn()
        pose_sampler = GripperCamPoseSampler(inital_pos, initial_orn,
                                             theta_limits=[2.36, 3.93],
                                             r_limits=[0.05, 0.1],
                                             h_limits=[-0.05*s, 0.05*s],
                                             trans_limits=[-0.05*s, 0.05*s],
                                             yaw_limits=[-0.087*s, 0.087*s],
                                             pitch_limit=[-0.087*s, 0.087*s],
                                             roll_limit=[-0.087*s, 0.087*s])
        rec = SimpleRecorder(env, save_dir=save_dir)
    
        for i in tqdm(range(max_steps)):
            pos, orn = pose_sampler.sample_pose()
            action = dict(motion=(pos,orn,1), ref="abs")
            obs, rew, done, info = env.step(action)
            rec.step(obs, action, None, rew, done, info)
            
        rec.save()
        
        del env
        time.sleep(.5)
        print(f"{seed}/{value['num_episodes']}", save_dir)

# View Recorded Data

Show the dataset that we have recorded.

TODO: include orientation in the relative distance computation.

In [ ]:
from flow_control.servoing.playback_env_servo import PlaybackEnvServo

def get_recordings(directory):
    return sorted([os.path.join(directory, rec) for rec in os.listdir(directory) if os.path.isdir(os.path.join(directory, rec))])

demo_recordings = get_recordings(root_dir)
#goal_recordings = get_recordings(goal_dir)

recordings = demo_recordings
print("Number of recordings:", len(recordings))
print(recordings[0])
print(recordings[-1])

In [ ]:
# Load the demonstration episodes
playbacks = [PlaybackEnvServo(rec) for rec in recordings[:2]]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Layout

demo_index = 0
t_tcp_cam = playbacks[demo_index][0].cam.get_extrinsic_calibration()

positions = []
orientations = []
orn_inv = []
for frame_index in range(len(playbacks[demo_index])):
    t_tcp_robot = playbacks[demo_index][frame_index].robot.get_tcp_pose()
    trf = t_tcp_robot @ t_tcp_cam
    trf = np.linalg.inv(trf)
    positions.append(trf[0:3,3])
    orientations.append(R.from_matrix(trf[:3,:3]))
    orn_inv.append(R.from_matrix(trf[:3,:3]).inv())

positions = np.array(positions)
orientations = np.array(orientations)
orn_inv = np.array(orn_inv)
dist = np.linalg.norm(positions[:, None, :] - positions[None, :, :], axis=-1)

dist_orn = np.zeros((len(orientations), len(orientations)))
idx_x, idx_y = np.triu_indices(len(orientations))
orn_diff = (orientations[idx_x]*orn_inv[idx_y])
orn_diff = [x.magnitude() for x in orn_diff]
dist_orn[idx_x, idx_y] = orn_diff
dist_orn = dist_orn + dist_orn.T - np.diag(np.diag(dist_orn))

dist_cmb = dist/np.max(dist) + dist_orn/np.max(dist_orn)

# Plot the demonstrations
%matplotlib notebook
fig, ax = plt.subplots(1, 2,figsize=(8, 6))
fig.suptitle("Demonstration Frames")
ax[0].set_axis_off()
ax[1].set_axis_off()
image_h = ax[0].imshow(playbacks[0].cam.get_image()[0])

ax[1].imshow(dist-dist.T)
np.fill_diagonal(dist, 1e3)
np.fill_diagonal(dist_orn, 1e3)
np.fill_diagonal(dist_cmb, 1e3)

def update(demo_index, frame_index):
    image = playbacks[demo_index][frame_index].cam.get_image()[0]
    print("closest neighbor:", np.argmin(dist_cmb[frame_index]), "(frame index)")
    image_h.set_data(image)
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=0, max=len(playbacks)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=200-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)

## Light Version: w/o flowcontrol dependencies

In [ ]:
def get_recordings(directory):
    return sorted([os.path.join(directory, rec) for rec in os.listdir(directory) if os.path.isdir(os.path.join(directory, rec))])

root_dir = "/home/argusm/CLUSTER/robot_recordings/pose_estimation/sim_objects/random_views"
demo_recordings = get_recordings(root_dir)

recordings = demo_recordings
print("Number of recordings:", len(recordings))
print(recordings[0])
print(recordings[-1])

In [ ]:
from glob import glob

def get_image(demo_dir, frame_index, depth=False):
    arr = np.load(os.path.join(demo_dir, f"frame_{frame_index:06d}.npz"))
    rgb_gripper = arr["rgb_gripper"]
    return rgb_gripper

def get_reward(demo_dir):
    frame_names = sorted(glob(f"{demo_dir}/frame_*.npz"))
    rew = np.load(frame_names[-1])["rew"].item()
    return rew

def get_len(demo_dir):
    frame_names = sorted(glob(f"{demo_dir}/frame_*.npz"))
    return len(frame_names)

def get_info(demo_dir, frame_index):
    arr = np.load(os.path.join(demo_dir, f"frame_{frame_index:06d}.npz"), allow_pickle=True)
    return arr["info"].item()

def pos_orn_to_matrix(pos, orn):
    mat = np.eye(4)
    if isinstance(orn, np.quaternion):
        orn = np_quat_to_scipy_quat(orn)
        mat[:3, :3] = R.from_quat(orn).as_matrix()
    elif len(orn) == 4:
        mat[:3, :3] = R.from_quat(orn).as_matrix()
    elif len(orn) == 3:
        mat[:3, :3] = R.from_euler('xyz', orn).as_matrix()
    mat[:3, 3] = pos
    return mat

def get_tcp_pose(demo_dir, frame_index):
    arr = np.load(os.path.join(demo_dir, f"frame_{frame_index:06d}.npz"),allow_pickle=True)
    state = arr["robot_state"].item()
    return pos_orn_to_matrix(state["tcp_pos"], state["tcp_orn"])

def get_extr_cal(demo_dir):
    camera_info = np.load(Path(demo_dir) / "camera_info.npz", allow_pickle=True)
    extr = camera_info["gripper_extrinsic_calibration"]
    return extr
    
#fi = 3
#t_tcp_cam1 = playbacks[0][fi].cam.get_extrinsic_calibration()
#t_tcp_cam2 = get_extr_cal(recordings[0])
#pose1 = playbacks[0][fi].robot.get_tcp_pose()
#pose2 = get_tcp_pose(recordings[0], fi)
#diff = pose1 - pose2
#print(diff)
#print(t_tcp_cam1 -t_tcp_cam2)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import widgets, interact, Layout

demo_index = 40
demo_dir = recordings[demo_index]

t_tcp_cam = get_extr_cal(demo_dir)

positions = []
orientations = []
orn_inv = []
for frame_index in range(get_len(demo_dir)):
    t_tcp_robot = get_tcp_pose(demo_dir, frame_index)
    trf = t_tcp_robot @ t_tcp_cam
    trf = np.linalg.inv(trf)
    positions.append(trf[0:3,3])
    orientations.append(R.from_matrix(trf[:3,:3]))
    orn_inv.append(R.from_matrix(trf[:3,:3]).inv())

positions = np.array(positions)
orientations = np.array(orientations)
orn_inv = np.array(orn_inv)
dist = np.linalg.norm(positions[:, None, :] - positions[None, :, :], axis=-1)

dist_orn = np.zeros((len(orientations), len(orientations)))
idx_x, idx_y = np.triu_indices(len(orientations))
orn_diff = (orientations[idx_x]*orn_inv[idx_y])
orn_diff = [x.magnitude() for x in orn_diff]
dist_orn[idx_x, idx_y] = orn_diff
dist_orn = dist_orn + dist_orn.T - np.diag(np.diag(dist_orn))

dist_cmb = dist/np.max(dist) + dist_orn/np.max(dist_orn)

# Plot the demonstrations
%matplotlib notebook
fig, ax = plt.subplots(1, 2,figsize=(8, 6))
fig.suptitle("Demonstration Frames")
ax[0].set_axis_off()
ax[1].set_axis_off()
image_h = ax[0].imshow(get_image(demo_dir, 0))

ax[1].imshow(dist-dist.T)
np.fill_diagonal(dist, 1e3)
np.fill_diagonal(dist_orn, 1e3)
np.fill_diagonal(dist_cmb, 1e3)

def update(demo_index, frame_index):
    image = get_image(demo_dir, frame_index)
    print("closest neighbor:", np.argmin(dist_cmb[frame_index]), "(frame index)")
    image_h.set_data(image)
    fig.canvas.draw_idle()
    
slider_w = widgets.IntSlider(min=0, max=len(recordings)-1, step=1, value=0,
                             layout=Layout(width='70%'))
slider_i = widgets.IntSlider(min=0, max=200-1, step=1, value=0,
                             layout=Layout(width='70%'))

interact(update, demo_index=slider_w, frame_index=slider_i)